# Downloads all posters from Indico

This script used Selenium to fetch all the info from Indico.

Make sure you have downloaded the Firefox geckodriver and it is in your `PATH`.

Also make sure you have exported your SSO password as env variable `SSO_PWD` and your SSO username as `SSO_USER` before starting this notebook.

To download geckodriver:

- Go to: https://github.com/mozilla/geckodriver/releases
- Download: geckodriver-v0.26.0-macos.tar.gz
- Place the geckodriver in some folder and make sure that folder is in your PATH.

In [225]:
import time
import random
import os
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import json
import csv

Some settable variables:

In [233]:
# Debug flag
_debug = False

# Name of the output json file
_out_posters_json_file = 'posters.json'
_out_posters_csv_file = 'posters.csv'

# Where all posters will be dowloaded
_poster_dir = 'posters/'

# How log to max wait
_timeout = 30

In [3]:
# uploaded_posters_ids = []
all_posters = {}
all_posters['posters'] = []

## Open Session
Open an new webdriver session, go to Indico and sign in.

In [253]:
# Open indico
driver = webdriver.Firefox()
driver.get("https://indico.fnal.gov/event/19348/manage/contributions/") # url for our conference
# assert "INDICO" in driver.title

# Log-in via SSO
sso_btn = driver.find_element_by_class_name('external-provider-shib-sso')
sso_btn.click()

continue_btn = driver.find_element_by_class_name('ping-button')
continue_btn.click()

inputUser = driver.find_element_by_id('username')
inputUser.clear()
inputUser.send_keys(os.getenv('SSO_USER'))

inputPassword = driver.find_element_by_id('password')
inputPassword.clear()
inputPassword.send_keys(os.getenv('SSO_PWD'))
inputPassword.send_keys(Keys.RETURN)

# Wait for completion
time.sleep(3)

Next two cells was me playing around, we don't need to execute it.

In [243]:
%%bash -c : # This prevents cell from being executed

poster_table = driver.find_element_by_css_selector('table')
poster_table_lines = poster_table.find_elements_by_css_selector('tr')
# postertable[20].get_attribute('innerHTML')
# postertable[3].find_elements_by_tag_name('td')[3].get_attribute('innerHTML')
print('Number of posters', len(poster_table_lines))

poster_table_lines[1].find_elements_by_tag_name('td')[3].get_attribute('innerHTML')

cell = poster_table_lines[568].find_elements_by_tag_name('td')[6]
print(cell.get_attribute('innerHTML'))
cell.find_element_by_class_name('person-row').get_attribute('innerHTML')

cell = poster_table_lines[568].find_elements_by_tag_name('td')[11]
print(cell.get_attribute('innerHTML'))
material = cell.find_element_by_class_name('icon-attachment')
material.click()

In [244]:
%%bash -c : # This prevents cell from being executed
material_table = driver.find_element_by_class_name('tree')
material_table_lines = material_table.find_elements_by_css_selector('tr')
# postertable[20].get_attribute('innerHTML')
print('Number of materials', len(material_table_lines))
for row in material_table_lines:
    # The file name is the zero index td cell:
    cell = row.find_elements_by_tag_name('td')[0]
    print(cell.text)
    if '.pdf' not in cell.text:
        print('Not a PDF file!')
    material_link = cell.find_element_by_css_selector("[href]").get_attribute('href')
    print(material_link)
btn = driver.find_elements_by_class_name('ui-dialog-titlebar-close')
btn[0].click()

In [245]:
def find_poster(materials):
    '''
    Tries to find the poster pdf among all the 
    uploaded documents
    '''
    
    # Select PDF files only
    m = {k: v for k, v in materials.items() if '.pdf' in k}
    
    if not len(m):
        return 'NotFound', 'NotFound'
    
    # If there are more than 1 PDFs, select the one
    # that contains the word 'poster'
    if len(m) > 1:
        for k, v in m.items():
            if 'poster' in k.lower():
                return k, v
    
    # Otherwise return the first one
    return list(m.keys())[0], list(m.values())[0]

def find_video(materials):
    '''
    Tries to find the poster pdf among all the 
    uploaded documents
    '''
    
    extensions = ['.MOV', '.MPEG4', '.MP4', '.AVI', '.WMV', '.MPEGPS', '.FLV']
    
    for k, v in materials.items():
        for e in extensions:
            if e in k.upper():
                return k, v
            
    return 'NotFound', 'NotFound'


def get_materials_link():
    '''
    With a material pop-up window opened,
    this function reads all the materials uploaded
    and picks the first PDF file found.
    TODO: do we want the first? Maybe need to define
    another criterion
    
    returns:
    - link to poster file
    - link to video file
    - all links
    '''
    
    # Wait for completion
    try:
        element_present = EC.presence_of_element_located((By.CLASS_NAME, 'icon-folder-plus'))
        WebDriverWait(driver, _timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")

    material_table = driver.find_element_by_class_name('tree')
    material_table_lines = material_table.find_elements_by_css_selector('tr')
    
#     material_link = 'NotFound'
#     file_name = 'NotFound'

#     print('Number of materials', len(material_table_lines))
    
    all_materials = {}
    all_links = []
    
    for row in material_table_lines:
        # The file name is the zero index td cell:
        cell = row.find_elements_by_tag_name('td')[0]
        try:
            material_link = cell.find_element_by_css_selector("[href]").get_attribute('href')
            all_materials[cell.text] = material_link
            all_links.append(material_link)
        except:
            print('WARNING: Problem finding href element for', cell.text)
        
        
#         if _debug: print(cell.text)
#         if '.pdf' in cell.text:
#             file_name = cell.text
#             material_link = cell.find_element_by_css_selector("[href]").get_attribute('href')
#             if _debug: print(material_link)
#             break
    
    poster_file_name, poster_material_link = find_poster(all_materials)
    video_file_name, video_material_link = find_video(all_materials)
    
    return poster_material_link, video_material_link, all_links

Get the table with all the materials, and also get all the lines of this table.

In [255]:
poster_table = driver.find_element_by_css_selector('table')
poster_table_lines = poster_table.find_elements_by_css_selector('tr')

Loop over all the contribution table lines, and for each of them get:
- (element 1): the id
- (element 3): the link to the poster page
- (element 6): the presenters name
- (element 11): the link to the materials

In [258]:
def get_poster_info(poster_page):
    
    primary_authors = []
    co_authors = []
    
    driver.execute_script("window.open('" + poster_page + "');")
    driver.switch_to.window(driver.window_handles[1])
    
    try:
        element_present = EC.presence_of_element_located((By.ID, 'field-8814'))
        WebDriverWait(driver, _timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        return primary_authors, co_authors, '', '', '', -1
        
    speaker_lists = driver.find_elements_by_class_name('speaker-list')
#     print(len(speaker_lists))
    
    # Primary Authors
    for element in speaker_lists[1].find_elements_by_tag_name('a'):
        primary_authors.append(element.find_element_by_tag_name('span').get_attribute('innerHTML'))
        
    # Co-Authors
    if len(speaker_lists) > 2:
        for element in speaker_lists[2].find_elements_by_tag_name('a'):
            co_authors.append(element.find_element_by_tag_name('span').get_attribute('innerHTML'))
            
    # Mini-abstract
    mini_abstract = driver.find_element_by_id('field-8814').find_element_by_tag_name('p').get_attribute('innerHTML')
#     print(mini_abstract)
    
    # Collaboration
    other_fields = driver.find_elements_by_class_name('other-fields')
    try:
        collaboration = other_fields[0].find_element_by_tag_name('td').get_attribute('innerHTML')
        collaboration = collaboration.replace('\n', '')
        collaboration = collaboration.replace(' ', '')
#         print('collaboration:', collaboration) 
    except:
        collaboration = ''
        
    # Abstract url
    abstract_url = driver.find_element_by_class_name('icon-file-pdf').get_attribute('href')
    
    # Constrib session
    try:
        poster_session = driver.find_element_by_class_name('contrib-session').find_element_by_tag_name('a').get_attribute('innerHTML')
        poster_session = int(poster_session.lower().replace('poster session ', ''))
    except:
        poster_session = -1
        
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    
    return primary_authors, co_authors, mini_abstract, collaboration, abstract_url, poster_session
    
# print(get_poster_info('https://indico.fnal.gov/event/19348/contributions/186406/'))
# print(get_poster_info('https://indico.fnal.gov/event/19348/contributions/186356/'))



In [ ]:
all_posters['posters'] = []

driver.switch_to.window(driver.window_handles[0])

# current_line = 0

for i in range(current_line, len(poster_table_lines)):

    current_line = i

    row = poster_table_lines[i]

    id = -1
    poster_page = None
    title = 'None'
    presenters = 'None'
    poster_link = 'None'
    video_link = 'None'
    all_links = []
    primary_authors = []
    co_authors = []
    mini_abstract = ''
    collaboration = ''
    abstract_url = ''
    
    for j, cell in enumerate(row.find_elements_by_tag_name('td')):
#         print(cell.text)
        #
        # Poster ID
        #
        if j == 1:
            try:
                id = cell.find_element_by_class_name('vertical-aligner').get_attribute('innerHTML')
                print('Fetching Poster ID:', id)
            except:
                id = -2
                print('WARNING: Cannot find id name for')
        #
        # Poster Page Link and Title
        #
        if j == 3:
            try:
                poster_page = cell.find_element_by_css_selector("[href]").get_attribute('href')
                title = cell.find_element_by_css_selector("[href]").get_attribute('innerHTML')
            except Exception as err:
                print('WARNING: Cannot find poster page link:', err)
        #
        # Presenters
        #
        if j == 6:
            try:
                presenters = cell.find_element_by_class_name('person-row').get_attribute('innerHTML')
            except:
                presenters = 'None'
                print('WARNING: Cannot find author name for')
#             print('Presenters:', presenters)
        #
        # Materials
        #
        if j == 11:
            material = cell.find_element_by_class_name('icon-attachment')
            if material.text == 'None':
                link = 'None'
            else:
                material.click()
                poster_link, video_link, all_links = get_materials_link()
                close_btn = driver.find_elements_by_class_name('ui-dialog-titlebar-close')
                close_btn[0].click()
                
#             print('Link:', poster_link)
      
    # 
    # Authors, Miniabstract, collaboration, abstract url, poster_session
    #
    if poster_page is not None:
        primary_authors, co_authors, mini_abstract, collaboration, abstract_url, poster_session = get_poster_info(poster_page)
        
            
    if presenters is not 'None':
        all_posters['posters'].append({'id': id,
                                       'title': title,
                                       'presenters': presenters,
                                       'primary_authors': ','.join(primary_authors),
                                       'co_authors': ','.join(co_authors),
                                       'collaboration': collaboration,
                                       'mini_abstract': mini_abstract,
                                       'poster_link': poster_link,
                                       'video_link': video_link,
                                       'all_links': ','.join(all_links),
                                       'abstract': 'null',
                                       'track': 'null',
                                       'category': 'null',
                                       'abstract_url': abstract_url,
                                       'poster_session': poster_session
                                      })
    
    # Wait for completion
    time.sleep(0.5)

    if (i > 10 and _debug):
        break

Fetching Poster ID: 592


## Write to JSON file

In [204]:
with open(_out_posters_json_file, 'w') as outfile:
    json.dump(all_posters, outfile, indent=4)

## Write to CSV file

In [205]:
# now we will open a file for writing 
data_file = open(_out_posters_csv_file, 'w') 

# create the csv writer object 
csv_writer = csv.writer(data_file)

# Counter variable used for writing  
# headers to the CSV file 
count = 0
  
for element in all_posters['posters']: 

    if count == 0: 
        # Writing headers of CSV file 
        header = element.keys() 
        csv_writer.writerow(header) 
        count += 1
  
    # Writing data of CSV file 
    csv_writer.writerow(element.values()) 
  
    
data_file.close()

## Download all posters

In [57]:
os.system(f'mkdir -p {_poster_dir}')

n_bad = 0
n_total = 0

for p in all_posters['posters']:
    poster_id = p['id']
    link = p['poster_link']
    
    n_total += 1
        
    if int(poster_id) < 0:
        print('No poster id!')
        continue
        
    if link == "None" or link == "NotFound":
        print('No link for poster with id', poster_id)
        n_bad += 1
        continue
        
    print('Downloading poster with id', poster_id)

    response = requests.get(link)
    with open(f'{_poster_dir}/poster_id_{poster_id}.pdf', 'wb') as f:
        f.write(response.content)

No link for poster with id 4
No link for poster with id 5
No link for poster with id 6
No link for poster with id 8
No link for poster with id 9
No link for poster with id 11
No link for poster with id 12
No link for poster with id 13
No link for poster with id 14
No link for poster with id 15
No link for poster with id 16
No link for poster with id 17
No link for poster with id 18
No link for poster with id 19
No link for poster with id 20
No link for poster with id 21
No link for poster with id 23
No link for poster with id 26
No link for poster with id 27
No link for poster with id 28
No link for poster with id 29
No link for poster with id 35
No link for poster with id 36
No link for poster with id 38
No link for poster with id 40
No link for poster with id 42
No link for poster with id 44
No link for poster with id 46
No link for poster with id 47
No link for poster with id 48
No link for poster with id 49
No link for poster with id 50
No link for poster with id 51
No link for pos

No link for poster with id 303
No link for poster with id 304
No link for poster with id 305
No link for poster with id 306
No link for poster with id 307
No link for poster with id 308
No link for poster with id 309
No link for poster with id 310
No link for poster with id 311
No link for poster with id 312
No link for poster with id 313
No link for poster with id 314
No link for poster with id 315
No link for poster with id 316
No link for poster with id 317
No link for poster with id 318
No link for poster with id 319
No link for poster with id 320
No link for poster with id 321
No link for poster with id 322
No link for poster with id 323
No link for poster with id 325
No link for poster with id 326
No link for poster with id 327
No link for poster with id 328
No link for poster with id 329
No link for poster with id 330
No link for poster with id 331
No link for poster with id 332
No link for poster with id 333
No link for poster with id 335
No link for poster with id 336
No link 

No link for poster with id 575
No link for poster with id 576
No link for poster with id 578
No link for poster with id 580
No link for poster with id 581
No link for poster with id 582
No link for poster with id 583
No link for poster with id 584
No link for poster with id 585
No link for poster with id 586
No link for poster with id 587
No link for poster with id 588
No link for poster with id 589
No link for poster with id 590
No link for poster with id 591
No link for poster with id 592
No link for poster with id 593
No link for poster with id 594
No link for poster with id 595
No link for poster with id 609
No link for poster with id 610
No link for poster with id 611
No link for poster with id 612
No link for poster with id 613
No link for poster with id 616
No link for poster with id 617
No link for poster with id 618
No link for poster with id 619
No link for poster with id 620
No link for poster with id 622
No link for poster with id 623
No link for poster with id 625
No link 

In [59]:
print(n_bad, 'posters don\'t have links out of', n_total, ' (', float(n_bad/n_total*100.), ' %)')

553 posters don't have links out of 587  ( 94.20783645655877  %)
